<h1><center><b>Tropical Fishes</b></center></h1>
<h1><center><b>Baseline Training</b></center></h1>
<center><b>15 March 2023</b></center><br/>
<center><b>ISNCC IEEE @HBKU</b></center><br/>
<center><b>Nauman Ullah Gilal</b></center><br/>
<center><b>Co-Authors: Fahad Majeed, Khaled Al-Thelaya, Mehak Khan, Jens Schneider, and Marco Agus</b></center><br/>
<center><b>IDEAL Lab</b></center><br/>
<center><b>College of Science and Engineering</b></center><br/>
<center><b>Hamad Bin Khalifa University<b></center><br/></h4>


In [ ]:
!nvidia-smi

# Data Preparation
Check training data

In [ ]:
!ls scrapped_dataset_new

In [ ]:
!ls test

# train_path = '/workspace/Nauman/MENA/MENA-v1-4C-Gym-Scheme/MENA-v1-c-109/MENA-v1-c-109/Training'
# test_path = '/workspace/Nauman/MENA/MENA-v1-4C-Gym-Scheme/MENA-v1-c-109/MENA-v1-c-109/Testing'

# Data set Categories, Classes, Labels

In [ ]:
import os
categories = os.listdir('scrapped_dataset_new')
categories

In [ ]:
TRAIN_DIR = "scrapped_dataset_new"

# Visualization of training samples

In [ ]:
# Visualize the training data
import matplotlib.pyplot as plt
import numpy as np
rows = 2
cols = 5
fig, ax = plt.subplots(rows, cols, figsize=(60,60))
fig.suptitle("Showing one random image from each training class", y=1.05, fontsize=40, weight = 'bold') # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = TRAIN_DIR
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    if food_selected == '.DS_Store':
        continue
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10,weight = 'bold', color = 'red', fontsize = 30)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()
# https://matplotlib.org/users/tight_layout_guide.html


# Visualization of samples from Test set 1 

In [ ]:
# Visualize the training data
import matplotlib.pyplot as plt
import numpy as np
rows = 2
cols = 5
fig, ax = plt.subplots(rows, cols, figsize=(60,60))
fig.suptitle("Showing one random image from each training class", y=1.05, fontsize=40, weight = 'bold') # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = 'test'
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    if food_selected == '.DS_Store':
        continue
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10,weight = 'bold', color = 'green', fontsize = 30)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()
# https://matplotlib.org/users/tight_layout_guide.html


# Declaration of Efficient Net family with Resolution, Epochs, Learning Rate, Batch size and Label smoothing values

In [ ]:
architecture = 'efficientnet-b0'
logfile = 'efficientnet-b0-MENA.csv'

## The tables
effnet_size = ({
    'efficientnet-b0':300,      # original resolution
    'efficientnet-b1':240,      # original resolution
    'efficientnet-b2':400,      # original resolution
    'efficientnet-b3':400,      # original resolution
    'efficientnet-b4':500,      # original resolution
    'efficientnet-b5':456,      # original resolution
    'efficientnet-b6':528,      # original resolution
    'efficientnet-b7':600,      # original resolution
    'efficientnet-lite0':224,
    'efficientnet-lite2':260,
    'efficientnet-lite4':384,
    'resnet18':224,
    'resnet34':224,
    'resnet50':224,
    'resnet101':224,
    'resnet152':224
})
batch_size = ({ 
    'efficientnet-b0':128,      # initially 160
    'efficientnet-b1':128,      #
    'efficientnet-b2':128,       #88 160
    'efficientnet-b3':16,       #
    'efficientnet-b4':16,       #
    'efficientnet-b5':14,       #
    'efficientnet-b6':16,
    'efficientnet-b7':8,
    'efficientnet-lite0': 160,
    'efficientnet-lite2': 160,
    'efficientnet-lite4':24,
    
    'resnet18':400,             # 3080: ok
    'resnet34':320,            # RTX 3090 rocz!
    'resnet50':256,             # 3080: ok
    'resnet101':96,
    'resnet152':64
})
learning_rate = ({
    'efficientnet-b0':5e-4,     # 2e-3: too large, 5e-4: too large
    'efficientnet-b1':2e-3,     # 1e-3: 82.53/93.9
    'efficientnet-b2':1e-3,     # 1e-3, 2e-3     
    'efficientnet-b3':2e-3,    
    'efficientnet-b4':2e-3,  
    'efficientnet-b5':1.5e-3,  
    'efficientnet-b6':4e-3,
    'efficientnet-b7':4e-3,
    'efficientnet-lite0': 5e-4,
    'efficientnet-lite2':1e-3,
    'efficientnet-lite4':5e-4,
    'resnet18':2.823e-02,
    'resnet34':-1,
    'resnet50':1.016e-02,
    'resnet101':-1,
    'resnet152':-1,
})
epochs = ({
    'efficientnet-b0':(1,100), # for initally top losses
    'efficientnet-b1':(6,20),
    'efficientnet-b2':(15,50),
    'efficientnet-b4':(5,25),
    'efficientnet-lite0': (15,80),
    'efficientnet-lite2': (5,25),
    'resnet18':(10,50),
    'resnet34':(10,50),
    'resnet50':(10,50),
    'resnet101':(10,50),
    'resnet152':(10,50)
})
label_smoothing = ({           # a = 0.00     a = 0.05     a = 0.10     a = 0.15     a = 0.20
    'efficientnet-b0':0.05,    #
    'efficientnet-b1':0.05,    #
    'efficientnet-b2':0.05,    # 0.05
    'efficientnet-b3':0.05,    #
    'efficientnet-b4':0.05,    #
    'efficientnet-b5':0.05,
    'efficientnet-b6':0.05,
    'efficientnet-b7':0.05,
    'efficientnet-lite0': 0.05,
    'efficientnet-lite2': 0.05,
    'resnet18':0.05,
    'resnet34':0.05,
    'resnet50':0.05,
    'resnet101':0.05,
    'resnet152':0.05
})
## Testing accuracies (top1,top5)
scores = ({
    'efficientnet-b0':(0,0),
    'efficientnet-b1':(0,0),
    'efficientnet-b2':(0,0),
    'efficientnet-b3':(0,0),
    'efficientnet-b4':(0,0),
    'efficientnet-lite0':(0,0),
    'efficientnet-lite2': (0,0),
    'resnet50':(0,0)
})
tta_scores = ({
    'efficientnet-b0':(0,0),
    'efficientnet-b1':(0,0),
    'efficientnet-b2':(0,0),
    'efficientnet-b3':(0,0),
    'efficientnet-b4':(0,0),
    'efficientnet-lite0':(0,0),
    'efficientnet-lite2': (0,0),
    'efficientnet-lite4':(0,0),
    'resnet50':(0,0)
})

 # Writing training data into csv

In [ ]:
from pathlib import Path
import os.path
train_dir = "scrapped_dataset_new"
training_path = Path(train_dir)

In [ ]:
import pandas as pd
filepaths = list(training_path.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
train_df = pd.DataFrame(images)
train_df

# Viualization of Training Data, Samples per class

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.facecolor'] = 'white'
lbl = train_df['Label']
# print(lbl)
# print(lbl.size)
plt.figure(figsize=(70,20))
ax = sns.countplot(x= lbl, data=train_df)
ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')

# Images Per Catergory

In [ ]:
counted = train_df.groupby(["Label"]).size()
print(counted)

# Test set with 50 Images per Class

In [ ]:
test_dir = "test"
test_path = Path(test_dir)
filepaths = list(test_path.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
test_df= pd.DataFrame(images)
test_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
lbl = test_df['Label']
# print(lbl)
# print(lbl.size)
plt.figure(figsize=(70,20))
ax = sns.countplot(x= lbl, data=test_df)
ax.bar_label(ax.containers[0], weight = 'bold', fontsize = '30', color = 'red')
plt.xticks(rotation=90, fontsize = 40, weight = 'bold', color = 'black')
plt.yticks( fontsize = 40, weight = 'bold', color = 'black')
plt.xlabel("Labels", fontsize = 40, weight = 'bold', color = 'black')
plt.ylabel("Number of Images", fontsize = 40, weight = 'bold', color = 'black')

# Import Fastai and pytorch Library

In [ ]:
from fastai.vision.all import *
from fastai import __version__
import torch
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
from fastai.vision import *
from fastai import optimizer, losses, metrics
from functools import partial, wraps
import matplotlib.pyplot as plt
import random

# Fastai version checking

In [ ]:
import fastai
fastai.__version__
torch.__version__

# Torch version checking

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch.cuda.is_available()


In [ ]:
# # batch size
# bs = 128
# # image resolution
# img_size = 224
model = EfficientNet.from_pretrained("efficientnet-b0", num_classes=10)

# Preparation of data for learner (Fastai learner)

In [ ]:
# defining the data augmentation 
# ds_tfms = ([RandTransform(tfm=RandomCrop (CropPad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True), 
#             RandTransform(tfm=TfmCoord (symmetric_warp), kwargs={'magnitude': (-0.2, 0.2)}, p=0.75, resolved={}, do_run=True, is_random=True), 
#             RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-40, 40)}, p=0.75, resolved={}, do_run=True, is_random=True), 
#             RandTransform(tfm=TfmAffine (flip_affine), kwargs={}, p=0.5, resolved={}, do_run=True, is_random=True), 
#             RandTransform(tfm=TfmAffine (zoom), kwargs={'scale': (1.0, 1.4), 'row_pct': (0, 1), 'col_pct': (0, 1)}, p=0.75, resolved={}, do_run=True, is_random=True), 
#             RandTransform(tfm=TfmLighting (brightness), kwargs={'change': (0.35, 0.65)}, p=0.75, resolved={}, do_run=True, is_random=True), 
#             RandTransform(tfm=TfmLighting (contrast), kwargs={'scale': (0.7, 1.43)}, p=0.75, resolved={}, do_run=True, is_random=True),
#             RandTransform(tfm=TfmCoord (jitter), kwargs={'magnitude': (-0.01, 0.01)}, p=0.3, resolved={}, do_run=True, is_random=True),
#             RandTransform(tfm=TfmCoord (skew), kwargs={'direction': (0, 7), 'magnitude': (0.2)}, p=0.75, resolved={}, do_run=True, is_random=True),
#             RandTransform(tfm=TfmAffine (squish), kwargs={'scale': (0.42, 2.4), 'row_pct': (0, 1), 'col_pct': (0, 1)}, p=0.75, resolved={}, do_run=True, is_random=True)], 
#            [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True)
#            ])

In [ ]:
#81.01% with less aug
# tfms = [Zoom (min_zoom=1.0, max_zoom=1.2, p=0.5, draw=None, draw_x=None,draw_y=None, size=None,mode='bilinear', pad_mode='reflection', batch=False,align_corners=False),
#         Rotate (max_deg=45, p=0.5 , mode='bilinear', pad_mode='reflection',align_corners=False, batch=False),
#         Warp (magnitude=0.2, p=0.75),
#         Flip (p=0.5, draw=None, size=None,mode='bicubic', pad_mode='reflection', align_corners=True,batch=False),
#         Brightness(max_lighting = 0.2, p = 0.75),
#         Contrast  (max_lighting = 0.2, p = 0.75),
#         Saturation(max_lighting = 0.2, p = 0.75)
    
#        ]

#-------------------------------------------------------------------------------------------------------------------

batch_tfms = [Zoom(),Rotate(), Flip(), Brightness(), Contrast(), Saturation()]


In [ ]:

# default augmentation
bs   = batch_size[architecture]
imgs = effnet_size[architecture]
resize = (imgs*4)//3
dls = (ImageDataLoaders.from_df(train_df, valid_pct=0.2,bs=bs,label_col=1,
                                        shuffle_train=True,
                                        item_tfms=Resize(imgs),
                                        batch_tfms = batch_tfms
                                  
#                                         batch_tfms=aug_transforms(mult = 3,
#                                                                   do_flip=True,
#                                                                   flip_vert=False,
#                                                                   xtra_tfms=tfms,
# #                                                                   max_rotate=40.0, 
# #                                                                   min_zoom=1.0, 
# #                                                                   max_zoom=1.4,
# #                                                                   max_lighting=0.35,
# #                                                                   max_warp=0.2, 
# #                                                                   p_affine=0.75,
# #                                                                   p_lighting=0.75, 
# #                                                                   xtra_tfms=tfms
#                                                                   size=imgs, 
# #                                                                   mode='bilinear',
# #                                                                   pad_mode='reflection', 
# #                                                                   align_corners=True, 
# #                                                                   batch=False,
# #                                                                   min_scale=1.0))
                                 ))

print("Image size=", imgs)
print("Batch size=", bs)
print("Architecture=", architecture)
print(resize)


In [ ]:
# # default augmentation  best 83.83 with mult
# bs   = batch_size[architecture]
# imgs = effnet_size[architecture]
# resize = (imgs*4)//3
# data0 = (ImageDataLoaders.from_df(train_df, valid_pct=0.2,bs=bs,label_col=1,
#                                         shuffle_train=True,
#                                         item_tfms=Resize(imgs),
#                                         batch_tfms=aug_transforms(mult = 2,
#                                                                   do_flip=True,
#                                                                   flip_vert=False,
#                                                                   max_rotate=10.0,
#                                                                   max_zoom=1.1,
#                                                                   max_warp=0.2,
#                                                                   p_affine=0.75,
#                                                                   max_lighting=0.2,
#                                                                   p_lighting=0.75,
#                                                                   xtra_tfms=None
#             )))

# print("Image size=", imgs)
# print("Batch size=", bs)
# print("Architecture=", architecture)
# print(resize)


In [ ]:
dls.show_batch()

In [ ]:
loss_func = losses.CrossEntropyLossFlat()
model = EfficientNet.from_pretrained(architecture, num_classes=10)
learner_type = Learner

# Learner definition

In [ ]:
best_pth = '1epoch'
checkpoints = SaveModelCallback(fname=best_pth,monitor='accuracy',comp=np.greater, with_opt=True)
# brainfreeze = BnFreeze()
learn = ( learner_type(dls, model,metrics=[accuracy],loss_func=loss_func,
                        cbs=[ShowGraphCallback(),checkpoints]).to_fp16())
print("Best pth is=", best_pth)

# How do we choose learning rates?
If the learning rate is too slow, training will take a lot of time. If the learning rate is too high, 
we will be jumping around the minimum loss, getting farther and farther from the minimum and never reach it.
So to start the experiment of finding a learning rate, we train the model while increasing the learning rate.
Then we plot the loss against the learning rate, and stop when the loss starts to explode.

In [ ]:
learn.lr_find()
learn.recorder.plot_lr_find()

In [ ]:
low_lr = 1e-4
lr = 1e-3
learn.fit_one_cycle(7 , lr_max = slice(low_lr, lr))

# Unfreeze the Learner
fast.ai freezes the pre-trained part when we create the model. So up to this point, 
we only trained the last layer block. Next, we will unfreeze the pre-trained part and train the whole model.

In [ ]:
learn.unfreeze()

In [ ]:
low_lr = 1e-4
lr = 1e-3
learn.fit_one_cycle(20, lr_max = slice(low_lr, lr))

# Test(%) on Test set 1 (50 Images per class)

In [ ]:
data_test = (ImageDataLoaders.from_df(test_df, valid_pct = 0.0, splitter=None, shuffle=False, label_col=1, item_tfms=Resize(imgs)))
preds = learn.get_preds(dl=data_test)
preds
print("length of preds[1]",len(preds[1]))
acc= accuracy(preds[0], preds[1])
print(" BaselineTop-1 Accuracy:", acc)

In [ ]:
err = {}
log_preds, y  = learn.tta(dl=data_test)
tta_acc = accuracy(log_preds, y)
print(tta_acc)
err[0] = (100.0, 100.0*(1.0-float(tta_acc)))

In [ ]:
inter_test = ClassificationInterpretation.from_learner(learn, dl =data_test)
inter_test.plot_confusion_matrix(figsize = (10,10))

In [ ]:
# PATH="models/tf_exp0_scrapped_dataset.p"
# torch.save(model.state_dict(), PATH)

# Saving best pth with baseline for future using and for scheme.

In [ ]:
best_pth

In [ ]:
# import shutil
# source = 'models/'+best_pth+'.pth'
# destination = 'models/baseline/'+best_pth
# # Copy the file from source to destination
# shutil.copy(source, destination)

In [ ]:
bs

In [ ]:
dls = (ImageDataLoaders.from_df(train_df, valid_pct = 0.0, bs=bs, splitter=None, shuffle=False, label_col=1, item_tfms=Resize(imgs)))

In [ ]:
entire_interp = ClassificationInterpretation.from_learner(learn, dl = dls)

In [ ]:
num_images = 0
for batch in dls.train:
    num_images += len(batch[0])
print("Number of images:", num_images)

In [ ]:
import csv

# Get the top losses and their corresponding indexes
losses, idx = entire_interp.top_losses(len(train_df), largest = False)

# Get the image paths, label, and loss values corresponding to the top losses
image_paths_and_labels_losses = [(train_df.iloc[int(i)]['Filepath'], train_df.iloc[int(i)]['Label'], losses[j].item()) for j, i in enumerate(idx)]

# Write the data to a CSV file
with open('min_losses.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Filepath', 'Label', 'Loss'])
    for path, label, loss in image_paths_and_labels_losses:
        writer.writerow([path, label, loss])

In [ ]:
import pandas as pd
min_losses = pd.read_csv('min_losses.csv')
min_losses

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = min_losses

# Group the samples by category and select the 50 samples with the minimum loss
min_loss_200_per_cat = df.loc[df.groupby('Label')['Loss'].nsmallest(200).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
min_loss_200_per_cat.to_csv('min_losses_200_per_cat.csv', index=False)
min_loss_200_per_cat

In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=min_loss_200_per_cat)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./min_losses_200_per_cat/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./min_losses_200_per_cat/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

In [ ]:
# NOW, WE APPLY HERE OCR THAT WILL DETECT THE IMAGES WITH TEXT, WE REMOVE THE TEXT IMAGES FORM 1500 MIN LOSS IMAGES 
# We are removing the images that contains text from min_toplosses dataframe that contains 1000 images
import pytesseract
# import pandas as pd
# from pathlib import Path
# import os.path
# from fastai.vision.all import *
def optical_character_recognition(file, path = True):
  """ Simple OCR of text from images.

  Parameters
  ----------
  file: Path or str
    Image to examine.
  path: bool
    Indicates whether the file passed in is a path to a file, or an already opened Pillow Image.
    
  Returns:
  ----------
  str
    Text that was detected.
  """
  if path == True:
    # Use Pillow's Image class to open the image
    img = Image.open(file)
    new_size = tuple(2*x for x in img.size)
    img = img.resize(new_size, Image.Resampling.LANCZOS)
    img = img.convert('L')
    text = pytesseract.image_to_string(img, lang='eng', config='-c tessedit_char_whitelist=01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz --psm 3 --oem 3')
    # Remove '\n\x0c' whiich is found in every image
    text = text[:-3]
  else: 
    # Already opened with Pillow
    new_size = tuple(2*x for x in file.size)
    img = file.resize(new_size, Image.Resampling.LANCZOS)
    img = img.convert('L')
    text = pytesseract.image_to_string(file, lang='eng', config='-c tessedit_char_whitelist=01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz --psm 3 --oem 3')
    # Remove '\n\x0c' whiich is found in every image
    text = text[:-3]

  return text

In [ ]:
def find_artificial_text(img_folder, edge_removal = True, verbose = False):
  """ Searches for artificial text in each image in a image folder using pytesseract.

  Parameters
  ----------
  img_folder: Path or str
    Folder with images to examine.
  edge_removal: bool
    Should the function attempt to crop the edges of the to remove detected text.
    If no text is found, the cropped image is saved.
  verbose: bool
    Should the function print out which images are cropped, and which images have been found to have text.  
    
  Returns:
  ----------
  DataFrame containing images marked as possessing artifical text. 
  """
  from tqdm.notebook import tqdm
  # Initialize DataFrame
  text_detected = []
  classes = ['Cleaner-Wrasse',
 'Broomtail-Wrasse',
 'Clarks-anemonefish',
 'Blackspot-Snapper',
 'Yellowtail-Surgeonfish',
 'Whaleshark',
 'Blackfin-Butterflyfish',
 'Ocellated-Eagle-Ray',
 'Marbled_Electric_Ray',
 'Longfin-Bannerfish']
  column_names = ['Filepath', 'Text Found']
  df_mt = pd.DataFrame(columns = column_names)
  for image_path in tqdm(sorted(img_folder.ls())):
    im = Image.open(image_path).convert('RGB')
    if edge_removal == True:
      # if the image has text
      if len(optical_character_recognition(im, path = False))>0:
        if verbose == True:print(f'Text detected in image {image_path.name}');
        # if the text is along the top or bottom edge, overwrite the old image else make note of it in a dataframe
        if len(optical_character_recognition((crop_image(image_path)).convert('RGB'), path = False)) == 0:
          if verbose == True:print(f'Cropping removed text in image {image_path.name}');
          crop_image(image_path).save(image_path)
        else:
          df_mt = df_mt.append({'Filepath':image_path,'Text Found':optical_character_recognition(image_path)}, ignore_index=True)
    # No cropping      
    else:
      if len(optical_character_recognition(im, path=False))>0:
        if verbose==True:print(f'Text detected in image {image_path.name}');
        df_mt = df_mt.append({'Filepath':image_path,'Text Found':optical_character_recognition(image_path)}, ignore_index=True)

  # Create dataframe containing images with artificial text towards the center
  if verbose == True:print(f'{len(df_mt)} images with text found in folder {img_folder}.');
  return df_mt

In [ ]:
classes = ['Cleaner-Wrasse',
 'Broomtail-Wrasse',
 'Clarks-anemonefish',
 'Blackspot-Snapper',
 'Yellowtail-Surgeonfish',
 'Whaleshark',
 'Blackfin-Butterflyfish',
 'Ocellated-Eagle-Ray',
 'Marbled_Electric_Ray',
 'Longfin-Bannerfish']

In [ ]:
# Lets look in a folder and grab a file
df_artificial = []
df = []
local_path = Path('min_losses_200_per_cat')
for i in range(len(classes)):
    img_folder = local_path/classes[i]
    df_artificial.append(find_artificial_text(img_folder, edge_removal = False, verbose = True))
    df.append(df_artificial[i])

In [ ]:
text_detected_df1 = pd.concat(df_artificial)
print(len(text_detected_df1))
print(text_detected_df1)

In [ ]:
arabic_txt_detected = text_detected_df1.to_csv('texted_images_from_min_loss_1500.csv', index = False)
arabic_txt_detected = pd.read_csv('texted_images_from_min_loss_1500.csv')
arabic_txt_detected

In [ ]:
min_loss_200_per_cat['new_path']=None
min_loss_200_per_cat.reset_index(drop=True,inplace=True)
min_loss_200_per_cat

In [ ]:
li=str(min_loss_200_per_cat.iloc[0]['Filepath']).split('/')
li[1]+'/'+li[2]

In [ ]:
for i in range(0,len(min_loss_200_per_cat)):
    li=str(min_loss_200_per_cat.iloc[i]['Filepath']).split('/')
    min_loss_200_per_cat.at[i,'new_path']=li[1]+'/'+li[2]


In [ ]:
arabic_txt_detected['new_path']=None
arabic_txt_detected.reset_index(drop=True,inplace=True)
arabic_txt_detected

In [ ]:
for i in range(0,len(arabic_txt_detected)):
    li=str(arabic_txt_detected.iloc[i]['Filepath']).split('/')
    arabic_txt_detected.at[i,'new_path']=li[1]+'/'+li[2]

In [ ]:
arabic_txt_detected

In [ ]:
min_loss = min_loss_200_per_cat
print("length of data is:",len(min_loss['new_path']))
txt_noisy_images = arabic_txt_detected
print("length of text detected is:",len(txt_noisy_images))
txt_noisy_images_lst = list(txt_noisy_images['new_path'])
# print(txt_noisy_images_lst)
text_free = min_loss[~min_loss['new_path'].isin(txt_noisy_images_lst)]
after_ocr = text_free.to_csv('text_free.csv', index = False)
after_ocr = pd.read_csv('text_free.csv')
after_ocr

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = after_ocr

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_100 = df.loc[df.groupby('Label')['Loss'].nsmallest(100).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_100.to_csv('per_cat_100.csv', index=False)
per_cat_100

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_100

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('1000.png', bbox_inches='tight')


In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_100)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./baseline_10_images_per_cat/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./baseline_10_images_per_cat/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

# 1-Select 50 per cat 

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_100

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_50 = df.loc[df.groupby('Label')['Loss'].nsmallest(50).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_50.to_csv('per_cat_50.csv', index=False)
per_cat_50

# 1-Display 50 images per cat

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_50

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('50.png', bbox_inches='tight')


# copy images to folder 

In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_5_inverse)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./per_cat_50/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./per_cat_50/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

# After select 50 images per cat with min losses- update the csv then go for 20 per cat

In [ ]:
per_cat_100

In [ ]:
aa = per_cat_100
aa
bb = per_cat_50
cc = list(bb['Filepath'])
updated_data = aa[~aa['Filepath'].isin(cc)]
updated_data

# 2-Select 20 images per cat with min losses

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = updated_data

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_20 = df.loc[df.groupby('Label')['Loss'].nlargest(20).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_20_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['Loss'] > condition].nsmallest(5, columns=['Loss'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_20.to_csv('per_cat_20.csv', index=False)
per_cat_20

# Display 20_cat per cat

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_20

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('20.png', bbox_inches='tight')


In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_20)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./per_cat_20/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./per_cat_20/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

# After Select 20 per cat update csv

In [ ]:
aa = updated_data
aa
bb = per_cat_20
cc = list(bb['Filepath'])
updated_data = aa[~aa['Filepath'].isin(cc)]
updated_data

# 3-Select 10 images per cat

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = updated_data

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_10 = df.loc[df.groupby('Label')['Loss'].nsmallest(10).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_10.to_csv('per_cat_10.csv', index=False)
per_cat_10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_10

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('10.png', bbox_inches='tight')


In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_10)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./per_cat_10/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./per_cat_10/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

# Update after selecting 10 images per cat

In [ ]:
aa = updated_data
aa
bb = per_cat_10
cc = list(bb['Filepath'])
updated_data = aa[~aa['Filepath'].isin(cc)]
updated_data

# 4-select 5 images per cat with min losses

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = updated_data

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_5 = df.loc[df.groupby('Label')['Loss'].nsmallest(5).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_5.to_csv('per_cat_5.csv', index=False)
per_cat_5

# Display 5 per cat

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_5

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('5.png', bbox_inches='tight')


In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_5)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./per_cat_5/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./per_cat_5/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

In [ ]:
aa = updated_data
aa
bb = per_cat_5
cc = list(bb['Filepath'])
updated_data = aa[~aa['Filepath'].isin(cc)]
updated_data

In [ ]:
per_cat_50

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_50

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_10_1 = df.loc[df.groupby('Label')['Loss'].nsmallest(10).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_10_1.to_csv('per_cat_10_1.csv', index=False)
per_cat_10_1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_10_1

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('per_cat_10_1.png', bbox_inches='tight')


In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_10_1)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./per_cat_10_1/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./per_cat_10_1/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

In [ ]:
aa = per_cat_50
aa
bb = per_cat_10_1
cc = list(bb['Filepath'])
updated_data = aa[~aa['Filepath'].isin(cc)]
updated_data

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = updated_data

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_5_2 = df.loc[df.groupby('Label')['Loss'].nsmallest(5).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_5_2.to_csv('per_cat_5_2.csv', index=False)
per_cat_5_2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_5_2

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('5_2.png', bbox_inches='tight')


In [ ]:
from glob import glob
import random
from tqdm.notebook import tqdm
import shutil

# Helper method to split dataset into train and test folders
def split_data(df:pd.core.frame.DataFrame=None,filepath=None,shuffle=True,perc=0.0):
    def get_class(file):
        return file.split('/')[-2].strip()
    
    if type(filepath)!=type(None):
        files = [file for file in glob(filepath+'/**/*.jpg')]
    else:
        assert(type(df)!=type(None))
        files = df['Filepath']
        
    classes = set([get_class(name) for name in files])
    
    D = { key : [] for key in classes }
    for file in tqdm(files):
        D[get_class(file)].append(file)
    xt, xv = [],[]    
    for key,val in D.items():
        keys = [key for n in range(len(val))]
        F = [(x,y) for x,y in zip(val,keys)]
        if shuffle: random.shuffle(F)            
        N = int(len(val)*perc+0.5)
        xv += F[:N]
        xt += F[N:]
        
    if shuffle:
        random.shuffle(xt)
        random.shuffle(xv)
        
    y_train = [label for _,label in xt]
    x_train = [f for f,_ in xt]
    y_valid = [label for _,label in xv]
    x_valid = [f for f,_ in xv]
    
    return (x_train,y_train),(x_valid,y_valid)
        
    
train, valid = split_data(df=per_cat_5_1)
!mkdir -p ./data_train
!mkdir -p ./data_valid
# create folders
for c in set(train[1]): 
    os.makedirs(f'./per_cat_5_2/{c}',exist_ok=True)
    os.makedirs(f'./data_valid/{c}',exist_ok=True)
for file,c in tqdm(zip(train[0],train[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./per_cat_5_2/{c}/{name}')
for file,c in tqdm(zip(valid[0],valid[1])):
    name = file.split('/')[-1]
    shutil.copyfile(file,f'./data_valid/{c}/{name}')
# !ls ./data_valid/bouzellouf/

In [ ]:
updated_data

In [ ]:
aa = updated_data
aa
bb = per_cat_5_3
cc = list(bb['Filepath'])
updated_data = aa[~aa['Filepath'].isin(cc)]
updated_data

In [ ]:
# Select 100 samples per class with min losses from all the data
# import pandas as pd

# Load the DataFrame containing filepath, label, and loss value
df = updated_data

# Group the samples by category and select the 50 samples with the minimum loss
per_cat_5_3 = df.loc[df.groupby('Label')['Loss'].nsmallest(5).index.get_level_values(1)]
# condition = 0.00002  # change the condition to whatever value you want
# fs_5_per_cat_after_ocr = df.loc[df.groupby('Label').apply(lambda x: x[x['los'] > condition].nsmallest(5, columns=['los'])).index.get_level_values(1)]


# Save the selected DataFrame as a CSV file
per_cat_5_3.to_csv('per_cat_5_3.csv', index=False)
per_cat_5_3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = per_cat_5_3

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('5_3.png', bbox_inches='tight')


In [ ]:
per_cat_50

In [ ]:
cat_rows = per_cat_50[per_cat_50['Label'] == 'Blackfin-Butterflyfish']
cat_rows

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Load the DataFrame containing filepath, label, and loss value
df = cat_rows

# Determine the number of rows and columns needed to display all the images
n = len(df)
ncols = 5
nrows = (n + ncols - 1) // ncols

# Calculate the figure size based on the number of rows and columns
figwidth = 20
figheight = nrows * figwidth / ncols

# Create a figure and subplot for each image
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figwidth, figheight))

# Plot each image
counter = 0
for i, row in df.iterrows():
    img = imread(row['Filepath'])
    ax = axs[counter // ncols][counter % ncols]
    ax.imshow(img)
    ax.set_title(f"Loss: {row['Loss']:.8f}\nLabel: {row['Label']}", color = 'red')
    ax.axis('off')
    counter += 1

# Save the plot as an image file
plt.savefig('butterflyfish.png', bbox_inches='tight')
